In [ ]:

import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.figure_factory as ff
from arch import arch_model

from ipywidgets import HBox, VBox, Dropdown, Output
from scipy.optimize import fmin, minimize
from scipy.stats import t
from scipy.stats import norm
from math import inf
from IPython.display import display
from scipy.linalg import sqrtm

import bs4 as bs
import requests
import yfinance as yf
import datetime

## Dynamic Conditional Correlation - What is it and why should you care?

In [ ]:
#returns lower matrix as vector by rows
def vecl(matrix): 
    lower_matrix = np.tril(matrix,k=-1)
    array_with_zero=np.matrix(lower_matrix).A1
   

    array_without_zero = array_with_zero[array_with_zero!=0]

    return array_without_zero

In [ ]:
# return cdf of standerdized residuals with nu deg of freedom as vector of length T

def garch_t_to_u(rets, res):
    mu = res.params['mu']
    nu = res.params['nu']
    est_r = rets - mu
    h = res.conditional_volatility
    std_res = est_r /np.sqrt(h) #perhaps just h as h is altready standard deviation
    udata = t.cdf(std_res, nu)
    return udata

In [ ]:
# return log likelihood of L_c
def loglike_norm_dcc_copula(theta, udata):
    N, T = np.shape(udata)
    llf = np.zeros((T,1))
    trdata = np.array(norm.ppf(udata).T, ndmin=2) # p-value of one-tail test on cdf of t-distirbution on standardized residuals
                                                  # ppf == oposite of cdf
    
    Rt, veclRt =  dcceq(theta,trdata)

    for i in range(0,T):
        llf[i] = -0.5* np.log(np.linalg.det(Rt[:,:,i]))
        if (i % 10) ==0:
            print('llf: {}, det {}, a is {} '.format(llf[i],np.linalg.det(Rt[:,:,i]), theta))
        llf[i] = llf[i] - 0.5 *  np.matmul(np.matmul(trdata[i,:] , (np.linalg.inv(Rt[:,:,i]) - np.eye(N))) ,trdata[i,:].T)
    llf = np.sum(llf)

    return -llf

In [ ]:

# calculate R_t and Q as in quation (1) return Rt and vech(R_t) - lower part of matrix as stacked vector 
def dcceq(theta,trdata):
    T, N = np.shape(trdata)

    a, b = theta
    
    if min(a,b)<0 or max(a,b)>1 or a+b > .999999:
        a = .9999 - b
        
    Qt = np.zeros((N, N ,T))

    Qt[:,:,0] = np.cov(trdata.T)
  

    Rt =  np.zeros((N, N ,T))
    veclRt =  np.zeros((T, int(N*(N-1)/2)))
    
    Rt[:,:,0] = np.corrcoef(trdata.T)
    print(Rt[:,:,0])
    
    for j in range(1,T):
        Qt[:,:,j] = Qt[:,:,0] * (1-a-b)
        Qt[:,:,j] = Qt[:,:,j] + a * np.matmul(trdata[[j-1]].T, trdata[[j-1]])
        Qt[:,:,j] = Qt[:,:,j] + b * Qt[:,:,j-1]
        Rt[:,:,j] = np.divide(Qt[:,:,j] , np.matmul(np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2)).T , np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2))))
    
    for j in range(0,T):
        veclRt[j, :] = vecl(Rt[:,:,j].T)
    return Rt, veclRt

In [ ]:
 #estimate univariate GARCH models and return parameters 
model_parameters = {}
udata_list = []

def run_garch_on_return(rets, udata_list, model_parameters):
    for x in rets:
        am = arch_model(rets[x], dist = 't')
        short_name = x.split()[0]
        model_parameters[short_name] = am.fit(disp='off')
        udata = garch_t_to_u(rets[x], model_parameters[short_name])
        udata_list.append(udata)
    return udata_list, model_parameters

## Initially run GARCH on the individual time series, and transform them to the uniform distribution

In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2010,1,1)
end = datetime.datetime(2020,12,30)
close_prices = yf.download(tickers, start=start, end=end)['Adj Close']

In [ ]:
rets = ((close_prices / close_prices.shift(1)) - 1 ).dropna(how='all') * 100


In [ ]:
udata_list = []
udata_list, model_parameters = run_garch_on_return(rets.iloc[:,:25].dropna(), udata_list, model_parameters)

In [ ]:
#udata_list
#rets

## Setup our DDC Model, and then run it on our 5 securities

In [ ]:
# lambda functions definds that the following ineq must hold 

cons = ({'type': 'ineq', 'fun': lambda x:  -x[0]  -x[1] +1})
bnds = ((0, 0.5), (0, 0.9997))

In [ ]:
cons['fun']

In [ ]:
%time opt_out = minimize(loglike_norm_dcc_copula, np.array([0.01, 0.95]), args = (udata_list,), bounds=bnds, constraints=cons)

In [ ]:
print(opt_out.success)
print(opt_out)

In [ ]:
opt_out

In [ ]:
udata_list

In [ ]:
llf  = loglike_norm_dcc_copula(opt_out.x, udata_list)
llf

In [ ]:
udata_list

In [ ]:
trdata = np.array(norm.ppf(udata_list).T, ndmin=2)
Rt, veclRt = dcceq(opt_out.x, trdata)


In [ ]:
stock_names = [x.split()[0] for x in rets.iloc[:,:50].columns]


In [ ]:
corr_name_list = []
for i, name_a in enumerate(stock_names):
    if i == 0:
        pass
    else:
        for name_b in stock_names[:i]:
            corr_name_list.append(name_a + "-" + name_b)

## ABBV (AbbVie) and A (Agilent Technologies) Look interesting

In [ ]:
dcc_corr = pd.DataFrame(veclRt, index = rets.iloc[:,:50].dropna().index, columns= corr_name_list)
dcc_plot = px.line(dcc_corr, title = 'Dynamic Conditional Correlation plot', width=1000, height=500)
dcc_plot.show()

In [ ]:
garch_vol_df = pd.concat([pd.DataFrame(model_parameters[x].conditional_volatility/100)*1600 for x in model_parameters], axis=1)
garch_vol_df.columns = stock_names

In [ ]:
px.line(garch_vol_df, title='GARCH Conditional Volatility', width=1000, height=500).show()


In [ ]:
px.scatter(garch_vol_df, x = 'ABBV', y='A', width=1000, height=500, title='GARCH Volatility').show()

In [ ]:
px.line(np.log((1+rets.iloc[:,:5].dropna()/100).cumprod()), title='Cumulative Returns', width=1000, height=500).show()

In [ ]:
rets.loc[:, ['ABBV','A']].corr()

In [ ]:
def update_corr_data(change):
    a1corr = rets.loc[:, pair_dropdown.value.split('-')].corr().values[0][1]
    a1dcc = pd.DataFrame(veclRt[:,corr_name_list.index(pair_dropdown.value)],index = rets.iloc[:,:5].dropna().index)
    a1dcc.columns = ['DCC']
    a1dcc['corr'] = a1corr
    corr_line_plot = px.line(a1dcc, title = 'DCC vs unconditional correlation for ' + pair_dropdown.value, width=1000, height=500)
    output_graphics.clear_output()
    with output_graphics:
        display(corr_line_plot)

In [ ]:
output_graphics = Output()
pair_dropdown = Dropdown(options=[''] + corr_name_list)
pair_dropdown.observe(update_corr_data, 'value')
VBox([pair_dropdown, output_graphics])

In [ ]:
import sympy as sp
from sympy import *

In [ ]:

print(np.sqrt(np.array(x, ndmin = 2 )))
np.array(x, ndmin = 2 )

In [ ]:
x = np.array([[ 4,  8,  9], ##reg. matrix
       [10, 10, 12],
       [13, 14, 15]])
x
y = np.matmul(x.T,x) #symetric and P.D.
y

In [ ]:
A = np.sqrt(np.array(np.diag(y), ndmin=2)).T 
B = np.sqrt(np.array(np.diag(y), ndmin=2))
np.divide (y , np.matmul(A, B))
A


In [ ]:
Z = Matrix([sp.sqrt(q1),sp.sqrt(q5),sp.sqrt(q9)])
Z * Z.T

In [ ]:
B_sqrt = B**(1/2)
B_sqrt_inv = B_sqrt.inv(method ="LU")
B_sqrt_inv * A * B_sqrt_inv

In [ ]:
var('q1,q2,q3,q4,q5,q6,q7,q8,q9')

A = Matrix([[q1,q2,q3],
             [q4,q5,q6],
             [q7,q8,q9]])
B = Matrix([[q1,0,0],
             [0,q5,0],
             [0,0,q9]])
C = B.inv(method = "LU")
print(C)
B*A

In [ ]:
Q = np.array([[q1,q2,q3],
             [q4,q5,q6],
             [q7,q8,q9]])
Q